Clase ComplejoSimplicial con sus métodos principales

In [232]:
from itertools import chain, combinations

class ComplejoSimplicial:

    """
    Inicializa el complejo simplicial tomando como argumento lo que serían los maximales del poset
    """
    def __init__(self, sim_maximales):
        self.simplices = dict()
        sim_derivados = self.derivar_simplices(sim_maximales)
        for sim in sim_derivados:
            if isinstance(sim, int):
                dim = 0
            else:    
                dim = len(sim)-1
            if dim not in self.simplices:
                self.simplices[dim] = set()
            self.simplices[dim].add(sim)

    def derivar_simplices(self, simplices_maximales):
        """
        Genera el complejo simplicial tomando todos los subconjuntos de los símplices maximales.
        
        Args:
        simplices_maximales (list of tuples): Lista de símplices maximales.
        
        Returns:
        set: El conjunto de todos los símplices en el complejo simplicial.
        """
        complejo = set()
        for simplejo in simplices_maximales:
            complejo.update(self.potencia(simplejo))
        return complejo
    
    def potencia(self, simplejo):
        """
        Genera todos los subconjuntos de un simplejo (incluido el propio simplejo).
        
        Args:
        simplejo (tuple): Un simplejo representado como una tupla de vértices.
        
        Returns:
        set: Conjunto de todos los subconjuntos del simplejo.
        """
        return set(chain.from_iterable(combinations(simplejo, r) for r in range(len(simplejo) + 1)))

    """
    Devuelve la dimension del complejo simplicial
    """
    def dim(self):
        return max(self.simplices.keys())

    """
    Devuelve una lista con las caras de un complejo simplicial para una dimension dada
    """
    def caras(self, dim):
        caras = []
        for cara in self.simplices[dim]:
            caras.append(cara)
        return caras

    """
    Devuelve todas las caras del complejo simplicial
    """
    def face_set(self):
        return set(cara for dim in self.simplices for cara in self.simplices[dim])

    """
    Devuelve la estrella de un simplice dado
    Se buscan las caras que contienen completamente a dicho simplice
    """
    def estrella(self, simplice):
        simplice = set(simplice)  # Convertimos el simplice en un conjunto(set), para poder hacer intersecciones
        return set(cara for cara in self.face_set() if simplice.intersection(cara) == simplice)
    
    """
    Devuelve la estrella cerrada de un simplice dado.
    La estrella cerrada es el menor subcomplejo simplicial que contiene a la estrella de un simplice dado.
    """
    def estrella_cerrada(self, simplex):
        # Obtenemos la estrella del simplex dado
        estrella = self.estrella(simplex)
        # Inicializamos la estrella cerrada como un conjunto vacío
        estrella_cerrada = set()
        cubiertos = set()  # Para rastrear qué simplices de la estrella están cubiertos
        
        # Recorremos todos los símplices de menor a mayor dimensión
        for dim in sorted(self.simplices.keys()):
            for cara in self.simplices[dim]:
                cara_set = set(cara)
                
                # Verificamos si la cara cubre algunos simplices de la estrella
                if any(set(s).issubset(cara_set) for s in estrella):
                    estrella_cerrada.add(cara)
                    # Actualizamos los simplices de la estrella que están cubiertos
                    cubiertos.update(s for s in estrella if set(s).issubset(cara_set))
                
                # Si ya hemos cubierto todos los simplices de la estrella, podemos detenernos
                if cubiertos == estrella:
                    break
        
        return estrella_cerrada
  
    """
    Devuelve el link del simplice dado
    El link de un simplice es el conjunto de simplices de la estrella cerrada con interseccion vacia con el simplice.
    """
    def link(self, simplice):
        # Obtenemos la estrella cerrada del simplice. Lo obtenemos como conjunto, lo pasamos a lista
        estrella_cer = self.estrella_cerrada(simplice)
        # Convertimos en complejo simplicial, para poder obtener todas sus caras
        ec = ComplejoSimplicial(estrella_cer)
        # Nos quedamos con los simplices de la estrella cerrada que no intersecan con el simplice del que queremos obtener el link
        return set(cara for cara in ec.face_set() if not set(simplice).intersection(cara))

    """
    Permite printear el complejo simplicial
    """
    def __str__(self):
        pass

    def print_caras(self):
        dim = self.dim()
        for i in range(0,dim+1):
            print("Caras de dim: ", i)
            d_caras = self.caras(i)
            for cara in d_caras:
                print(cara)
    
    """
    Calcula la característica de Euler del complejo simplicial
    """
    def carac_euler(self):
        """ dim = self.dim()
        res = 0
        for i in range(0, dim+1):
            if i % 2 == 0:
                res += len(self.simplices[dim])
            else:
                res -= len(self.simplices[dim])
        return res """
        # Forma más eficiente
        return sum((-1)**k * len(self.simplices[k]) for k in range(self.dim()+1))
    

    """
    Devuelve el número de componentes conexas del complejo simplicial
    """
    def num_componentes_conexas(self):
        n = 0
        vertices = list(self.simplices[0])
        aristas = self.simplices[1]

        v_conexos = []
        v_inconexos = []
        v0 = vertices[0]

        while True:
            #print('hola')
            for v in vertices[:]:
                print(f"v0: {v0}, v: {v}, existe_camino: {self.existe_camino(v0[0], v[0], aristas)}")
                if self.existe_camino(v0[0], v[0], aristas):
                    v_conexos.append(v)
                    vertices.remove(v)
                else:
                    v_inconexos.append(v)
            
            n += 1
            if len(vertices) == 0:
                return n
            v0 = vertices[0]
            v_conexos = []
            v_inconexos = []


    def existe_camino(self, v, u, aristas):
        #print(aristas)
        # Crear el grafo como un diccionario de adyacencia
        grafo = dict()
        
        # Construir el diccionario de adyacencia a partir de las aristas
        for v1, v2 in aristas:
            if v1 not in grafo.keys():
                grafo[v1] = []
            if v2 not in grafo:
                grafo[v2] = []
            grafo[v1].append(v2)
            grafo[v2].append(v1)  # Como es un grafo no dirigido, añadir ambas conexiones

        # Función recursiva para la búsqueda en profundidad (DFS)
        def bep(vertice, visitados):
            if vertice == u:
                return True
            visitados.append(vertice)  # Marcar el vértice como visitado
            # Recorrer los vecinos
            if vertice in grafo.keys():
                for vecino in grafo[vertice]:  # Obtener vecinos del vértice
                    if vecino not in visitados:  # Solo visitar los no visitados
                        if bep(vecino, visitados):
                            return True
            
            return False
        
        return bep(v, list())
    
    def skeleton(self, k):
        """
        Devuelve el esqueleto de dimensión k del complejo simplicial.
        
        Args:
        k (int): La dimensión máxima de los simplices en el esqueleto.
        
        Returns:
        ComplejoSimplicial: Un nuevo complejo simplicial que es el esqueleto de dimensión k.
        """
        # Recopilar todas las caras de dimensión k o menor
        caras_k_o_menor = set()
        for dim in range(k + 1):
            if dim in self.simplices:
                caras_k_o_menor.update(self.simplices[dim])
        
        # Crear un nuevo complejo simplicial con estas caras
        return ComplejoSimplicial(list(caras_k_o_menor))
    


EJEMPLO 1: Tetraedro

In [233]:
# Definimos un complejo simplicial
sc = ComplejoSimplicial([(0,1,2,3)])

In [234]:
# Conjunto de todas las caras del complejo simplicial
sc.print_caras()

Caras de dim:  0
(0,)
(1,)
(2,)
(3,)
Caras de dim:  1
(0, 1)
(1, 2)
(0, 3)
(2, 3)
(0, 2)
(1, 3)
Caras de dim:  2
(0, 1, 2)
(0, 1, 3)
(1, 2, 3)
(0, 2, 3)
Caras de dim:  3
(0, 1, 2, 3)


In [235]:
# Dimensión del complejo simplicial
sc.dim()

3

In [236]:
# Vértices del complejo simplicial
sc.caras(0)

[(0,), (1,), (2,), (3,)]

In [237]:
# Aristas del complejo simplicial
sc.caras(1)

[(0, 1), (1, 2), (0, 3), (2, 3), (0, 2), (1, 3)]

In [238]:
# 2-símplices del complejo simplicial
sc.caras(2)

[(0, 1, 2), (0, 1, 3), (1, 2, 3), (0, 2, 3)]

In [239]:
# 3-símplices del complejo simplicial
sc.caras(3)

[(0, 1, 2, 3)]

In [240]:
# Característica de Euler (4-6+4=2)
sc.carac_euler()

1

In [241]:
# Estrella de la arista (0,1)
sc.estrella([0,1])

{(0, 1), (0, 1, 2), (0, 1, 2, 3), (0, 1, 3)}

In [242]:
# Estrella cerrada de la arista (0,1)
# Como el total está en la estrella, la estrella cerrada claramente es el total
sc.estrella_cerrada([0,1])

{(0, 1), (0, 1, 2), (0, 1, 2, 3), (0, 1, 3)}

In [243]:
# Link de la arista (0,1)
sc.link([0,1])

{(), (2,), (2, 3), (3,)}

In [244]:
# Componentes conexas
sc.num_componentes_conexas()

v0: (0,), v: (0,), existe_camino: True
v0: (0,), v: (1,), existe_camino: True
v0: (0,), v: (2,), existe_camino: True
v0: (0,), v: (3,), existe_camino: True


1

EJEMPLO 2: Borde del tetraedro

In [245]:
sc1=sc.skeleton(2)

In [246]:
# Conjunto de todas las caras del complejo simplicial
sc1.print_caras()

Caras de dim:  0
(0,)
(1,)
(2,)
(3,)
Caras de dim:  1
(0, 1)
(1, 2)
(0, 3)
(2, 3)
(0, 2)
(1, 3)
Caras de dim:  2
(0, 1, 2)
(0, 1, 3)
(1, 2, 3)
(0, 2, 3)


In [247]:
sc1.dim()

2

In [248]:
# Estrella del vértice 0
sc1.estrella((0,))

{(0,), (0, 1), (0, 1, 2), (0, 1, 3), (0, 2), (0, 2, 3), (0, 3)}

In [249]:
sc1.estrella_cerrada((0,))

{(0,), (0, 1), (0, 1, 2), (0, 1, 3), (0, 2), (0, 2, 3), (0, 3)}

In [250]:
# Link del vértice 0
sc1.link((0,))

{(), (1,), (1, 2), (1, 3), (2,), (2, 3), (3,)}

In [251]:
# Característica de Euler 
sc1.carac_euler()

2

In [252]:
# Componentes conexas
sc1.num_componentes_conexas()

v0: (0,), v: (0,), existe_camino: True
v0: (0,), v: (1,), existe_camino: True
v0: (0,), v: (2,), existe_camino: True
v0: (0,), v: (3,), existe_camino: True


1

EJEMPLO 3

In [255]:
sc2 =ComplejoSimplicial([(0,1),(1,2,3,4),(4,5),(5,6),(4,6),(6,7,8),(8,9)])

In [256]:
# Conjunto de todas las caras del complejo simplicial
sc2.print_caras()

Caras de dim:  0
(6,)
(2,)
(5,)
(8,)
(4,)
(1,)
(7,)
(0,)
(3,)
(9,)
Caras de dim:  1
(0, 1)
(2, 4)
(1, 2)
(3, 4)
(6, 8)
(4, 6)
(1, 4)
(2, 3)
(6, 7)
(4, 5)
(8, 9)
(5, 6)
(1, 3)
(7, 8)
Caras de dim:  2
(1, 2, 3)
(1, 3, 4)
(6, 7, 8)
(2, 3, 4)
(1, 2, 4)
Caras de dim:  3
(1, 2, 3, 4)


In [257]:
sc.dim()

3

In [259]:
# 1-esqueleto
sc2.skeleton(1).print_caras()

Caras de dim:  0
(6,)
(2,)
(5,)
(8,)
(4,)
(1,)
(7,)
(0,)
(3,)
(9,)
Caras de dim:  1
(0, 1)
(2, 4)
(1, 2)
(3, 4)
(6, 8)
(4, 6)
(1, 4)
(2, 3)
(6, 7)
(4, 5)
(8, 9)
(5, 6)
(1, 3)
(7, 8)


In [260]:
# Estrella del vértice 4
sc2.estrella((4,))

{(1, 2, 3, 4),
 (1, 2, 4),
 (1, 3, 4),
 (1, 4),
 (2, 3, 4),
 (2, 4),
 (3, 4),
 (4,),
 (4, 5),
 (4, 6)}

In [261]:
# Link del vértice 4
sc2.link((4,))

{(), (1,), (1, 2), (1, 2, 3), (1, 3), (2,), (2, 3), (3,), (5,), (6,)}

In [262]:
# Característica de Euler
sc2.carac_euler()

0

In [263]:
# Componentes conexas
sc2.num_componentes_conexas()

v0: (6,), v: (6,), existe_camino: True
v0: (6,), v: (2,), existe_camino: True
v0: (6,), v: (5,), existe_camino: True
v0: (6,), v: (8,), existe_camino: True
v0: (6,), v: (4,), existe_camino: True
v0: (6,), v: (1,), existe_camino: True
v0: (6,), v: (7,), existe_camino: True
v0: (6,), v: (0,), existe_camino: True
v0: (6,), v: (3,), existe_camino: True
v0: (6,), v: (9,), existe_camino: True


1

EJEMPLO 4

In [264]:
sc3 = sc2.skeleton(1)

In [265]:
sc3.carac_euler()

-4

EJEMPLO 5

EJEMPLO 6

In [266]:
sc6 = ComplejoSimplicial([(1,2,4),(1,3,6),(1,4,6),(2,3,5),(2,4,5),(3,5,6)])

In [267]:
sc6.face_set()

{(),
 (1,),
 (1, 2),
 (1, 2, 4),
 (1, 3),
 (1, 3, 6),
 (1, 4),
 (1, 4, 6),
 (1, 6),
 (2,),
 (2, 3),
 (2, 3, 5),
 (2, 4),
 (2, 4, 5),
 (2, 5),
 (3,),
 (3, 5),
 (3, 5, 6),
 (3, 6),
 (4,),
 (4, 5),
 (4, 6),
 (5,),
 (5, 6),
 (6,)}

In [268]:
sc6.dim()

2

In [269]:
sc6.estrella((1,4))

{(1, 2, 4), (1, 4), (1, 4, 6)}

In [271]:
sc6.link((1,4))

{(), (2,), (6,)}

In [272]:
sc6.carac_euler()

0

Hay hasta ejemplo 11